In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.datasets import make_blobs
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.optim as optim
import torch.nn.functional import F
from torch.utils.data import DataLoader
import torchvision
from torchvision import transforms
import time
import matplotlib.pyplot as plt

In [ ]:
data = [1.0,2.0,3.0]
tensor = torch.Tensor(data)
print("1D Data: ")
print(tensor)

data = [[1., 2., 3.], [4.,5., 6.]]
tensor = torch.Tensor(data)
print("2D Data: ")
print(tensor)

In [ ]:
rand = torch.randn(2,3,3)
rand

In [ ]:
X = np.arange(0, 10, 0.5)
epsilon = np.random.normal(0, 0.5, len(X))
y = (2 + 3*X)

plt.scatter(X, y, s=20)
plt.title("Randomly Generated Data")
plt.show()

In [ ]:
X, y = X.reshape(-1,1), y.reshape(-1,1)

model = LinearRegression()
model.fit(X, y)
print("W_1:", model.coef_[0][0])
print("W_0:", model.intercept_[0])

In [ ]:
torch.seed()

class SGDRegression(nn.Module):
    def __init__(self):
        self.__init__()
        self.fc1 = nn.Linear(1,1)
    
    def forward(self,x):
        y = self.fc1(x)
        return y

model = SGDRegression()
loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(),lr = 0.01)
num_epochs = 5000

x_train  = torch.tensor(X,dtype = torch.float32)
y_train = torch.Tensor(y,dtype = torch.float32)



In [ ]:
for name,param in model.named_parameters():
    print(name,param)

In [ ]:
losses = []
for epoch in range(num_epochs):
    y_pred = model(x_train)
    loss = loss_fn(y_pred,y_train)
    losses.append(y_pred,y_train)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [ ]:
for name,param in model.named_parameters():
    print(name,param)

## Logistic Regression

In [ ]:
centres = [[1,1],[-1,-1]]

x,y = make_blobs(n_samples=750,centers=centres, cluster_std = 0.8, random_state=0)

plt.scatter(x[:,0], x[:,1],c=y)
plt.title("Dummy Classification Data")
plt.plot([-2,2],[2,-2],"r")
plt.show()


y = y.reshape(-1,1)

In [ ]:
torch.seed()

class LogisticRegression(nn.Module):
    def __init__(self):
        super.__init__()
        self.fc1 = nn.Linear(2,1)
    
    def forward(self,x):
        y = torch.sigmoid(self.fc1(x))
        return y

model = LogisticRegression()
loss_fn = torch.nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(),lr = 0.01)
num_epochs = 5000

x_train = torch.tensor(x,dtype = torch.float32)
y_train = torch.tensor(y,dtype = torch.float32)

In [ ]:
losses = []
for epoch in range(num_epochs):
    y_pred = model(x_train)
    loss = loss_fn(y_pred,y_train)
    losses.append(loss.items())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [ ]:
accuracy = torch.sum((y_pred>0.5).int()== y_train) / len(y_train)
accuracy

In [ ]:
train = torchvision.datasets.MNIST('../../data', train=True, download=True,
                      transform=transforms.Compose([ ### CONVERT ARRAY TO TENSOR
                          transforms.ToTensor()
                       ]))

test = torchvision.datasets.MNIST('../../data', train=False, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor()
                       ]))


print(train)

In [ ]:
for image,label in train:
    print(image.shape)
    print(label)
    break

counter = 0
for image,label in train:
    if counter==6:
        break
    image = image.squeeze().numpy()
    print("Image Label:",label)
    plt.imshow(image,cmap ="gray")
    plt.show()
    counter +=1

In [ ]:
batch_size = 128
loss_fn = nn.CrossEntropyLoss()
lr = 0.008
num_epochs = 5

In [ ]:
trainset = DataLoader(train, batch_size = batch_size,shuffle = True)
testset = DataLoader(test,batch_size = batch_size,shuffle = False)

for image,label in trainset:
    print(image.shape)
    print(label)
    break

In [ ]:
class FCNet(nn.Module):
    def __init__(self):
        super.__init__()
        self.fc1 = nn.Linear(28*28,64)
        self.fc2 = nn.Linear(64,64)
        self.fc3 = nn.Linear(64,64)
        self.fc4 = nn.Linear(64,10)

    def forward(self,x):
        x = x.view(-1,28*28)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

In [ ]:
model = FCNet()

optimizer = optim.Adam(model.parameters(),lr = lr)

start = time.time()
for epoch in range(1,num_epochs+1):
    print (f"Epoch {epoch}")
    losses = []
    for data in trainset:
        x,y = data
        optimizer.zero_grad()
        output = model(x)
        loss = loss_fn(output,y)
        losses.append(loss.item())
        loss.backward()
        optimizer.step()
    print(f"Loss: {np.mean(losses)}")
print(f"Time on CPU for Model Training: {time.time() - start}")

In [ ]:
correct = 0
total = 0

with torch.no_grad():
    for data in testset:
        x,y = data
        output = model(x)

        for idx, i in enumerate(output):
            if torch.argmax(i)==y[idx]:
                correct+=1
            total +=1
print("Accuracy: ", round(correct/total,2))


## GPU Acceleration

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

In [ ]:
model = FCNet().to(device)

optimizer = optim.Adam(model.parameters(),lr=lr)

start = time.time()
for epoch in range(1,num_epochs):
    print(f"Epoch {epoch}")
    losses = []
    for x,y in trainset:
        x,y = x.to(device) , y.to(device)
        optimizer.zero_grad()
        output = model(x)
        loss = loss_fn(output,x)
        losses.append(loss.item())
        loss.backward()
        optimizer.step()
    print(f"Loss: {np.mean(losses)}")
print("Time on GPU for Model Training: {time.time()-start}")


In [ ]:
model = FCNet().to(device)

optimizer = optim.Adam(model.parameters(), lr = lr)

num_epochs = 20
training_avg_losses = []
validation_avg_losses = []

for epoch in range(1,num_epochs+1):
    print(f"Epoch {epoch}")
    training_losses = []
    validation_losses = []

    model.train()
    for x,y in trainset:
        x,y = x.to(device), y.to(device)
        optimizer.zero_grad()
        output = model(x)
        loss = loss_fn(output,y)
        training_losses.append(loss.item())
        loss.backward()
        optimizer.step()
    
    model.eval()
    for x,y in testset:
        with torch.no_grad():
            x,y = x.to(device) , y.to(device)
            output = model(x)
            loss = loss_fn(output,y)
            validation_losses.append(loss.item())

    training_loss_avg = np.mean(training_losses)
    validation_loss_avg = np.mean(validation_losses)

    training_avg_losses.append(training_loss_avg)
    validation_avg_losses.append(validation_loss_avg)
    print(f"Training Loss: {training_loss_avg}")
    print(f"Validation Loss: {validation_loss_avg}")

In [ ]:
x = list(range(20))
plt.plot(x,training_avg_losses,label = "Training Curve")
plt.plot(x,validation_avg_losses, label = "Validation Curve")
plt.legend()
plt.show()